# Transfer learning **VGG16**

In [ ]:
pip install split-folders


In [ ]:
import splitfolders

# dataset path
input_folder = '/content/drive/MyDrive/SPAM IMAGE dataset/'

# Split with a ratio. To only split into training & validation set, set a tuple 'ratio', i.e,
# Train(70%), Val(20), Test(10)
splitfolders.ratio(input_folder, output='/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2',
                   seed=42, ratio=(.7,.2,.1),
                   group_prefix=None)

Copying files: 1751 files [00:31, 55.81 files/s] 


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Define your dataset directory paths
train_dir = '/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2/train'
validation_dir = '/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2//val'

In [ ]:
# Define image size and batch size
img_size = (224, 224)
batch_size = 32
# img_size = (224, 224) defines the size to which the input image is resized. Many pre-trained models (like VGG, ResNet, etc.) expect images of this size. These dimensions are widely used in various pre-trained models due to their effectiveness in capturing features while balancing computational cost. It's not necessary to define it explicitly in the provided code, as the preprocess_image function in the code already resizes the image to (224, 224).

# batch_size = 32 determines the number of samples/images propagated through the network at once. It affects the speed and performance of the training process. However, in the provided code (for image prediction in Flask), batch size isn't used explicitly since the prediction is performed on single images uploaded via the Flask app, not in a batched manner during training or inference.

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 1225 images belonging to 2 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 349 images belonging to 2 classes.


In [ ]:
# Load pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)


In [ ]:
# Combine base model with new classification layers
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
38/38 [==============================] - 286s 7s/step - loss: 0.1984 - accuracy: 0.9690 - val_loss: 0.0743 - val_accuracy: 0.9750
Epoch 2/10
38/38 [==============================] - 22s 579ms/step - loss: 0.0717 - accuracy: 0.9824 - val_loss: 0.0577 - val_accuracy: 0.9875
Epoch 3/10
38/38 [==============================] - 23s 589ms/step - loss: 0.0680 - accuracy: 0.9832 - val_loss: 0.0503 - val_accuracy: 0.9875
Epoch 4/10
38/38 [==============================] - 21s 565ms/step - loss: 0.0638 - accuracy: 0.9832 - val_loss: 0.0472 - val_accuracy: 0.9875
Epoch 5/10
38/38 [==============================] - 21s 562ms/step - loss: 0.0589 - accuracy: 0.9858 - val_loss: 0.0624 - val_accuracy: 0.9812
Epoch 6/10
38/38 [==============================] - 21s 562ms/step - loss: 0.0523 - accuracy: 0.9841 - val_loss: 0.0465 - val_accuracy: 0.9875
Epoch 7/10
38/38 [==============================] - 22s 567ms/step - loss: 0.0502 - accuracy: 0.9860 - val_loss: 0.0469 - val_accuracy: 0.9875
E

In [ ]:
# Save the model
model.save('spam_image_filter_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
# Load the trained model
model = tf.keras.models.load_model('spam_image_filter_model.h5')

In [ ]:
# Function to preprocess uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [ ]:
# Function to predict whether the image contains spam or not
def predict_spam(image_path):
    processed_img = preprocess_image(image_path)
    prediction = model.predict(processed_img)
    return prediction[0][0]  # Return the prediction value (probability)

# Provide the path to the uploaded image for prediction
uploaded_image_path = '/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2/test/NaturalImages/zzz_00806_9b3d519e0c_m.jpg'

# Predict whether the uploaded image contains spam
prediction_result = predict_spam(uploaded_image_path)

# Threshold for considering if it's spam or not (adjust as needed)
spam_threshold = 0.5

if prediction_result >= spam_threshold:
    print("The uploaded image is predicted as SPAM.")
else:
    print("The uploaded image is predicted as NOT SPAM.")


1/1 [==============================] - 0s 29ms/step
The uploaded image is predicted as NOT SPAM.


# **Resnet50**

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
# Define your dataset directory paths
train_dir = '/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2/train'
validation_dir = '/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2//val'


In [ ]:
# Define image size and batch size
img_size = (224, 224)
batch_size = 32

In [ ]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 1225 images belonging to 2 classes.


In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 349 images belonging to 2 classes.


In [ ]:
# Load pre-trained VGG16 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
# Add new classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Combine base model with new classification layers
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Freeze layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
38/38 [==============================] - 34s 704ms/step - loss: 0.5740 - accuracy: 0.7083 - val_loss: 0.4386 - val_accuracy: 0.8156
Epoch 2/10
38/38 [==============================] - 23s 612ms/step - loss: 0.3832 - accuracy: 0.8961 - val_loss: 0.3108 - val_accuracy: 0.9187
Epoch 3/10
38/38 [==============================] - 24s 637ms/step - loss: 0.2883 - accuracy: 0.9204 - val_loss: 0.2648 - val_accuracy: 0.9344
Epoch 4/10
38/38 [==============================] - 24s 639ms/step - loss: 0.2583 - accuracy: 0.9321 - val_loss: 0.2498 - val_accuracy: 0.9281
Epoch 5/10
38/38 [==============================] - 24s 629ms/step - loss: 0.2269 - accuracy: 0.9472 - val_loss: 0.2362 - val_accuracy: 0.9375
Epoch 6/10
38/38 [==============================] - 24s 629ms/step - loss: 0.2155 - accuracy: 0.9430 - val_loss: 0.2023 - val_accuracy: 0.9438
Epoch 7/10
38/38 [==============================] - 26s 676ms/step - loss: 0.2113 - accuracy: 0.9313 - val_loss: 0.2614 - val_accuracy: 0.8875

In [ ]:
# Save the model
model.save('spam_image_filter_modelresnet50.h5')


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('spam_image_filter_modelresnet50.h5')

# Function to preprocess uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to predict whether the image contains spam or not
def predict_spam(image_path):
    processed_img = preprocess_image(image_path)
    prediction = model.predict(processed_img)
    return prediction[0][0]  # Return the prediction value (probability)

# Provide the path to the uploaded image for prediction
uploaded_image_path = '/content/drive/MyDrive/SPAM IMAGE dataset/SPAM IMAGE dataset2/val/NaturalImages/zzz_00243_a8054e0b34_m.jpg'

# Predict whether the uploaded image contains spam
prediction_result = predict_spam(uploaded_image_path)

# Threshold for considering if it's spam or not (adjust as needed)
spam_threshold = 0.5

if prediction_result >= spam_threshold:
    print("The uploaded image is predicted as SPAM.")
else:
    print("The uploaded image is predicted as NOT SPAM.")


1/1 [==============================] - 1s 1s/step
The uploaded image is predicted as NOT SPAM.
